In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import norm
from scipy import stats
from scipy.integrate import quad
import pandas as pd
from scipy.stats import norm
from sklearn.linear_model import LinearRegression

# Case 5: Multi-Look and American Options

## Part 1: Asian Options

In [2]:
S0 = 100 
r = 0.02 
sigma = 0.15
T = 10  # Time horizon
n = 10  # Number of observations
n_sims = 100000  # Number of simulation paths
guarantee = 100
K = 100

In [3]:
# Exercise 1.1
def AsianOptionMonteCarlo(S0, T, r, sigma, N, M):
    dt = T / N 

    # Stock price paths
    S = np.zeros((M, N + 1))  
    S[:, 0] = S0  

    Z = np.random.normal(0, 1, (M, N)) 
    for t in range(1, N + 1):
        S[:, t] = S[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t - 1])

    # The new payoff function: max(average stock price - 100, 0)
    average_price = np.mean(S[:, 1:], axis=1)  
    payoff = np.maximum(average_price - 100, 0)

    discountedPrice = np.exp(-r * T) * np.mean(payoff)
    SE = np.exp(-r * T) * np.std(payoff) / np.sqrt(M)

    return discountedPrice, SE

asianOption_price, SE = AsianOptionMonteCarlo(S0, T, r, sigma, N=10, M=10000)

print("Price of the Asian contract (Monte Carlo): ", asianOption_price, "±", SE)

Price of the Asian contract (Monte Carlo):  16.00788017264252 ± 0.22771069737423946


In [4]:
simulations = [10000, 50000, 100000, 200000, 500000, 1000000]
results = []

numSteps=10

for M in simulations:
    price, se = AsianOptionMonteCarlo(S0, T, r, sigma, numSteps, M)
    results.append({"Number of Paths": M, "Price": price, "Standard Error": se})

# Convert results to DataFrame and display
results_df = pd.DataFrame(results)
results_df

,Number of Paths,Price,Standard Error
0,10000,15.693530,0.223390
1,50000,15.649505,0.101604
2,100000,15.654416,0.071537
3,200000,15.757483,0.050775
4,500000,15.807112,0.032113
5,1000000,15.808207,0.022796


In [5]:
# Exercise 1.2
def blackScholes(S0, r, sigma, T, K):

    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S0 / K) + (r - 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    
    phi = norm.cdf
    optionPrice = S0 * phi(d1) - np.exp(-r * T) * K * phi(d2)
    return optionPrice

print("Black-Scholes value of European call-option: ",blackScholes(S0, r, sigma, T=10, K=100))

Black-Scholes value of European call-option:  27.571349248747218


In [6]:
# Exercise 1.3
def deriveMoments(S0, r, sigma, T, N):   
    
    EQ_A = (1 / N) * sum(S0 * np.exp(r * np.arange(1, N + 1)))
    
    EQ_A2 = (1 / N**2) * sum(
        sum(S0**2 * np.exp(r * (i + np.arange(1, N + 1)) + sigma**2 * np.minimum(i, np.arange(1, N + 1))))
        for i in range(1, N + 1)
    )
    
    A0 = EQ_A / np.exp(r * T)
    b = np.sqrt((1 / T) * np.log(EQ_A2 / (A0**2)) - 2 * r)
    
    return A0, b

def asianOption_closedForm(S0, r, sigma, T, K, N):
    
    A0, b = deriveMoments(S0, r, sigma, T, N)
    print("A0: ", A0)
    print("b: ", b)

    d1 = (np.log(A0 / K) + (r + 0.5 * b**2) * T) / (b * np.sqrt(T))
    d2 = (np.log(A0 / K) + (r - 0.5 * b**2) * T) / (b * np.sqrt(T))
    
    phi = norm.cdf
    optionPrice = A0 * phi(d1) - np.exp(-r * T) * K * phi(d2)
    return optionPrice

print("Closed-form price of Asian option",asianOption_closedForm(S0, r, sigma, T, K, N = 10))


A0:  91.54399082959372
b:  0.09584030077451944
Closed-form price of Asian option 15.973823843043952


## Part 2: Unit-linked with continuous guarantee

**Remark:** We now have a continous guarantee, where the client has the right to "early exercise" the guarantee and walk away with value of 100 at any time 0 < t < 10

In [7]:
def binomialTreeContinuous(S0, guarantee, r, n, T, sigma):

    dt = T / n  
    R_dt = np.exp(r * dt)

    u = R_dt * np.exp(sigma * np.sqrt(dt))
    d = R_dt * np.exp(-sigma * np.sqrt(dt))

    p = (R_dt - d) / (u - d)

    # Stock prices 
    stock_prices = np.array([guarantee * (u ** j) * (d ** (n - j)) for j in range(n + 1)])
    
    # Payoff 
    contract_values = np.maximum(stock_prices, guarantee)
    
    # Backward induction 
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            continuation_value = (p * contract_values[j + 1] + (1 - p) * contract_values[j]) * np.exp(-r * dt)
            contract_values[j] = max(guarantee, continuation_value)  

    return contract_values[0]

print("Price of unit-linked with continous guarantee",binomialTreeContinuous(S0, guarantee, r, n, T, sigma))

Price of unit-linked with continous guarantee 111.90847226319141


In [8]:
def LSMC_unitLinked(S0, guarantee, r, T, sigma, M, N):
    dt = T / N  

    S = np.zeros((M, N + 1)) 
    S[:, 0] = S0  

    Z = np.random.normal(0, 1, (M, n)) 
    for t in range(1, N + 1):
        S[:, t] = S[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t - 1])

    payoffs = np.maximum(guarantee, S)  

    for t in range(N - 1, 0, -1):
        in_the_money = S[:, t] < guarantee 
            
        LR = LinearRegression().fit(
            S[in_the_money, t].reshape(-1, 1),  
            payoffs[in_the_money, t + 1] * np.exp(-r * dt)    
        )
        continuation_value = LR.predict(S[in_the_money, t].reshape(-1, 1))

        immediate_exercise_value = guarantee  

        exercise = immediate_exercise_value > continuation_value
        payoffs[in_the_money, t] = np.where(
            exercise,
            immediate_exercise_value,  
            payoffs[in_the_money, t + 1] * np.exp(-r * dt)    
        )

        payoffs[~in_the_money, t] = payoffs[~in_the_money, t + 1] * np.exp(-r * dt)  

    option_price = np.mean(payoffs[:, 1]) * np.exp(-r * dt)  
    SE = np.exp(-r * T) * np.std(payoffs[:, 1]) / np.sqrt(M)  
    return option_price, SE


unit_linked_price, SE = LSMC_unitLinked(S0, guarantee, r, T, sigma, M=10000, N=10)
print("Price of the unit-linked contract with continuous guarantee: ", unit_linked_price, "±", SE)


Price of the unit-linked contract with continuous guarantee:  111.44840853648545 ± 0.3355340592222646


In [10]:
simulations = [10000, 50000, 100000, 200000, 500000, 1000000]

results = []

for M in simulations:
    price, SE = LSMC_unitLinked(S0, guarantee, r, T, sigma, M, N=10)
    results.append({"Number of Paths": M, "Price": price, "Standard Error": SE})

df_results = pd.DataFrame(results)
df_results

,Number of Paths,Price,Standard Error
0,10000,111.718764,0.339685
1,50000,111.511998,0.151462
2,100000,111.444386,0.106781
3,200000,111.410727,0.075836
4,500000,111.557111,0.048230
5,1000000,111.473389,0.033997


In [ ]:
def binomialTree_final(S0, guarantee, r, n, T, sigma):

    dt = T / n  
    R_dt = np.exp(r * dt)

    u = R_dt * np.exp(sigma * np.sqrt(dt))
    d = R_dt * np.exp(-sigma * np.sqrt(dt))

    p = (R_dt - d) / (u - d)

    # Stock prices 
    stock_prices = np.array([guarantee * (u ** j) * (d ** (n - j)) for j in range(n + 1)])
    
    # Payoff 
    contract_values = np.maximum(stock_prices, guarantee)
    
    # Backward induction 
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            contract_values = (p * contract_values[j + 1] + (1 - p) * contract_values[j]) * np.exp(-r * dt)

    return contract_values[0]

print("Price of unit-linked with continous guarantee",binomialTree_final(S0, guarantee, r, n, T, sigma))